<font size="5">**Algorytmy danych geoprzestrzennych**</font><br>
<font size="4">Obliczenia geodezyjne</font>

<font size="4">Krzysztof Dyba</font>

# Współrzędne geograficzne

Współrzędne geograficzne to zestaw wartości liczbowych używany do określania położenia dowolnego punktu na powierzchni Ziemi. Dwoma podstawowymi składowymi współrzędnych geograficznych są:

- szerokość geograficzna (*latitude*; $\phi$) -- określa jak daleko na północ lub południe znajduje się punkt od równika Ziemi. Przyjmuje wartości z przedziału od 0° do 90° na północ (wartości dodatnie) i od 0° do 90° na południe (wartości ujemne) na biegunach.
- długość geograficzna (*longitude*; $\lambda$) -- określa jak daleko na wschód lub zachód znajduje się punkt od południka zerowego, który przebiega przez Greenwich w Anglii. Przyjmuje wartości z przedziału od 0° do 180° na wschód (wartości dodatnie) i od 0° do 180° na zachód (wartości ujemne).

Współrzędne geograficzne są zazwyczaj wyrażane w jednym z następujących formatów:

- stopnie, minuty i sekundy (*degrees*, *minutes*, *seconds*; DMS) -- stopnie reprezentują miarę kątową. Każdy stopień podzielony jest na 60 minut, a każda minuta na 60 sekund. Przykładowo: 52°17'34" N, 16°44'08" E.
- stopnie dziesiętne (*decimal degrees*; DD) -- współrzędne wyrażone są jako wartości dziesiętne stopni. Przykładowo: 52,2927° N, 16,7355° E.
- współrzędne planarne -- współrzędne wyrażone jako pomiary liniowe w układzie planarnym. Przykładowo: 345613,2393 m, 494269,4463 m (układ PUWG 1992, EPSG:2180).

## Konwersja

Wartość DMS konwertowana jest na stopnie dziesiętne przy użyciu następującego wzoru:

$$
\text{Stopnie dziesiętne} = \text{Stopnie} + \frac{\text{Minuty}}{60} + \frac{\text{Sekundy}}{3600}
$$

In [4]:
def dms_to_decimal(dms_str):
    
    # zamień znaki na spacje
    dms_str = dms_str.replace("°", " ").replace("'", " ").replace("\"", " ")
    # podziel tekst na części według spacji
    parts = dms_str.split()
    
    degrees = int(parts[0])
    minutes = int(parts[1])
    seconds = int(parts[2])
    direction = parts[3]
    
    decimal = degrees + (minutes / 60) + (seconds / 3600)
    decimal = round(decimal, 4)
    
    # sprawdź kierunek i nadaj znak
    if direction in ["S", "W"]:
        decimal = -decimal
    
    return decimal

In [5]:
print(dms_to_decimal("52°17'34\" N"), "°", sep = "")
print(dms_to_decimal("16°44'08\" E"), "°", sep = "")

52.2928°
16.7356°


Natomiast operacja odwrotna, czyli konwersja stopni dziesiętnych na wartość DMS, wymaga zastosowanie poniższych wzorów:

$$
\text{Stopnie} = \text{int}(\text{Stopnie dziesiętne})
$$

$$
\text{Minuty} = \text{int}\left((\text{Stopnie dziesiętne} - \text{Stopnie}) \times 60\right)
$$

$$
\text{Sekundy} = \left((\text{Stopnie dziesiętne} - \text{Stopnie}) \times 60 - \text{Minuty}\right) \times 60
$$

# Obliczenia

## Odległość

Dane są dwa punkty reprezentujące lokalizację Warszawy (21,0122°, 52,2296°) oraz Rzymu (12,5113°, 41,8919°).

### Euklidesowa

In [24]:
from qgis.core import QgsPointXY, QgsCoordinateReferenceSystem
from qgis.core import QgsCoordinateTransform, QgsCoordinateTransformContext

pt1 = (21.0122, 52.2296)
pt2 = (12.5113, 41.8919)

pt1 = QgsPointXY(pt1[0], pt1[1])
pt2 = QgsPointXY(pt2[0], pt2[1])

crs_4326 = QgsCoordinateReferenceSystem("EPSG:4326")
crs_3857 = QgsCoordinateReferenceSystem("EPSG:3857")

transform_context = QgsCoordinateTransformContext()
transform = QgsCoordinateTransform(crs_4326, crs_3857, transform_context)

pt1_3857 = transform.transform(pt1)
pt2_3857 = transform.transform(pt2)

distance = pt1_3857.distance(pt2_3857)
distance = round(distance / 1000, 2) # wynik w km

print("Odległość między Warszawą a Rzymem:", distance, "km")

Odległość między Warszawą a Rzymem: 1942.97 km


In [27]:
def euclidean_distance(pt1, pt2):
    import math
        
    # wyodrębnienie współrzędnych z obiektu QgsPoint
    x1, y1 = pt1.x(), pt1.y()
    x2, y2 = pt2.x(), pt2.y()
        
    distance = math.sqrt( (x2 - x1)**2 + (y2 - y1)**2 )
    distance = round(distance / 1000, 2) # wynik w km
    return distance

euclidean_distance(pt1_3857, pt2_3857)

1942.97

### Sferyczna

In [37]:
# https://en.wikipedia.org/wiki/Geographical_distance#Spherical-surface_formulae

def spherical_distance(pt1, pt2):
    from math import sin, cos, asin, sqrt, pi
    
    R = 6371009 # średni promień Ziemi w metrach
        
    coords = [pt1.x(), pt1.y(), pt2.x(), pt2.y()]
    # konwersja stopni na radiany
    coords = [coord * pi / 180 for coord in coords]
    x1, y1, x2, y2 = coords
    
    delta_x = cos(y2) * cos(x2) - cos(y1) * cos(x1)
    delta_y = cos(y2) * sin(x2) - cos(y1) * sin(x1)
    delta_z = sin(y2) - sin(y1)
    d_t = R * sqrt( delta_x**2 + delta_y**2 + delta_z**2 )
    distance = 2 * R * asin(d_t / (2 * R))
    distance = round(distance / 1000, 2)
    
    return distance

In [38]:
distance = spherical_distance(pt1, pt2)
print("Odległość między Warszawą a Rzymem:", distance, "km")

Odległość między Warszawą a Rzymem: 1315.51 km


### Elipsoidalna

In [22]:
from qgis.core import QgsDistanceArea

distance_area = QgsDistanceArea()
distance_area.setEllipsoid("WGS84")

distance = distance_area.measureLine(pt1, pt2)
distance = round(distance / 1000, 2) # wynik w km

print("Odległość między Warszawą a Rzymem:", distance, "km")

Odległość między Warszawą a Rzymem: 1316.2 km


## Powierzchnia

# Zadania:

14) Zaimplementuj funkcję `decimal_to_dms()` umożliwiającą konwersję stopni dziesiętnych na wartość DMS. Pamiętaj, aby zwrócona wartość posiadała odpowiedni kierunek geograficzny.
15) Zaimplementuj [wzór haversine](https://en.wikipedia.org/wiki/Haversine_formula#Formulation) i porównaj z innymi otrzymanymi wynikami na przykładzie odległości pomiędzy Warszawą a Rzymem. Wartości wyrażone w stopniach muszą zostać zamienione na radiany.

Notatki:

- odległość euklidesowa bazuje na twierdzeniu Pitagorasa,
- nie ma sensu używać odległości euklidesowej do obliczeń kątowych (w stopniach).